In [81]:
import numpy as np 
import pandas as pd

In [82]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold

In [83]:
df = pd.read_csv("GA_dataset.csv")

In [84]:
df.head()

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO


In [85]:
df.isnull().sum()

V1        0
V2        0
V3        0
V4        0
V5        0
Target    0
dtype: int64

In [86]:
df.replace("?", np.nan, inplace=True)
df.head()

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,NaN,NaN,4000.0,35.0,NEGATIVE,YES
3,NaN,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO


In [89]:
x = df.drop(columns=['Target'])
y = df['Target']

In [90]:
vt = VarianceThreshold(threshold=0.1)

In [91]:
num_pipe = ColumnTransformer(transformers=[(

    "Impute", SimpleImputer(strategy="mean"), [0,1,2,3]
)])

In [92]:
scale_pipe = Pipeline([
    ("Numeric", num_pipe),
    ("Scaler", StandardScaler())
])

In [93]:
cat_pipe = ColumnTransformer(transformers=[("Ordincal Encoding", OrdinalEncoder(), [4])])

In [94]:
cmplt_pipe = FeatureUnion([
    ("Numeric", scale_pipe),
    ("Categorical", cat_pipe)
])

In [95]:
pipe = Pipeline([
    ("Cp", cmplt_pipe),
    ("vt", vt)
])

In [96]:
pipe

Pipeline(steps=[('Cp',
                 FeatureUnion(transformer_list=[('Numeric',
                                                 Pipeline(steps=[('Numeric',
                                                                  ColumnTransformer(transformers=[('Impute',
                                                                                                   SimpleImputer(),
                                                                                                   [0,
                                                                                                    1,
                                                                                                    2,
                                                                                                    3])])),
                                                                 ('Scaler',
                                                                  StandardScaler())])),
                                                ('Categorical',
                                                 ColumnTransformer(transformers=[('Ordincal '
                                                                                  'Encoding',
                                                                                  OrdinalEncoder(),
                                                                                  [4])]))])),
                ('vt', VarianceThreshold(threshold=0.1))])

In [99]:
pipe.fit_transform(x).shape

(748, 4)

In [98]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [102]:
# use logistic regressor with default option for estimator
estimator = LogisticRegression()

# encode target via ordinal encoding
# reshaping y is necessary because the transformer only accepts column vector
y_encoded = OrdinalEncoder().fit_transform(y.to_numpy().reshape(-1,1).reshape(-1,1))

# create RFE object with necessary parameters
selectors = RFE(estimator, n_features_to_select=2)

# run RFE on dataset
selectors = selectors.fit(cmplt_pipe.fit_transform(x), y_encoded)

 #find the finally selected features
# selected features have 'True' in corresponding indices

print(selectors.support_)

[ True False  True False False]


/home/anjali/Desktop/MLP Course/menv/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/anjali/Desktop/MLP Course/menv/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/anjali/Desktop/MLP Course/menv/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/anjali/Desktop/MLP Course/menv/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-v